##  Deutsch–Jozsa on IBM quantum hardware

The Deutsch–Jozsa algorithm is one of the earliest examples of a quantum algorithm that outperforms classical solutions. <br> It shows how we can determine if a function is constant or balanced using just **one** query to a quantum oracle.

## The problem 

Given a function `f(x)` that returns either 0 or 1, and is guaranteed to be either:
- **Constant**: the same value for all inputs, or
- **Balanced**: returns 0 for half of the inputs and 1 for the other half,

Our task is to determine which type it is using **just one call** to the quantum oracle. You may wonder what the practical application of this is.<br> The truth is, it is not very practical on its own as an algorithm, but it is a stepping stone for other real-life applicable algorithms like Bernstein–Vazirani, Shor's and Simon's.


### STEP 1: Import our libraries 

Let's first begin by importing our libraries. 


In [ ]:
import numpy as np                             #needed for numerical operations
import matplotlib.pyplot as plt                #needed for plotting 

from qiskit import QuantumCircuit, transpile
    # QuantumCircuit: Defines and manipulates quantum circuits.  
    # transpile: Optimizes quantum circuits for a specific quantum backend.  
    # assemble: Converts the circuit into a runnable format for execution on simulators or real quantum hardware.

from qiskit.visualization import plot_histogram
    # Lets us visualize quantum measurement results in a histogram format.

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
    # QiskitRuntimeService: Lets us connect to a cloud provided quantum backend and execute our jobs on into
    # SamplerV2 Submits circuits to run on a backend and collects measurement results (bitstrings or counts).

### STEP 2: Choose our QPU backend

In [ ]:
service = QiskitRuntimeService(channel="ibm_quantum")
   # Connects to IBM Quantum service, uses your saved IBM token
   # the service object gives access to IBM Quantum devices and lets you run jobs on them.

for backend in service.backends(simulator=False): # fetch non-simulator backends, print out details
    print(f"{backend.name}: {backend.num_qubits} qubits, {backend.status().pending_jobs} jobs in queue, {backend.status().operational} operational")
    

# backend = service.backend("NAME_OF_CHOSEN_BACKEND")
backend = service.least_busy(simulator=False, operational=True) # or simply fetch the least busy 

### STEP 3: Define our circuit

#### 3.1 Create a quantum circuit with 3 qubits and 2 classical bits
  - **q0, q1**:   the input qubits (for the function input x)
  - **q2**:       the ancilla qubit (used to help encode the function output)
  - **c0, c1**:   classical bits to record measurement results

In [ ]:

qc = QuantumCircuit(3, 2)

#### 3.2  Prepare the ancillary qubit **q2** in state |1⟩ by applying an X gate to bit flip
- We'll use it to flip the phase based on f(x) later

qc.x(2)

#### 3.3 Apply Hadamard gates to all qubits
-  This puts the input qubits into superposition, so we can evaluate all possible inputs x at once

In [ ]:
qc.h([0, 1, 2]) 

 At this point:
 -  $q_0$ and $q_1$ are in a superposition of all possible input states:

      $$
      \frac{1}{2} (|00⟩ + |01⟩ + |10⟩ + |11⟩)  $$                 

     This means the circuit is exploring all possible inputs at the same time.
     
 -  $q_2$ (the ancilla) is in the |−⟩ state   
    $$\frac{|0⟩ - |1⟩}{\sqrt{2}}$$
 
     This special state enables something called **phase kickback**.<br>
     Phase kickback means that when we apply the oracle (CNOTs),
     the effect of the function $f(x)$ is not stored in the ancilla qubit itself, <br>
     but instead it flips the **sign (phase)** of the amplitude of the input state |x⟩. <br>
     Here, $|x⟩$ refers to one of the possible 2-qubit basis states like $|00⟩$, $|01⟩$, $|10⟩$, or $|11⟩$.  of $q_0$ and $q_1$ <br>

    This change affects the **entire superposition** of $q_0$ and $q_1$,  
    and it’s what allows the algorithm to detect whether $f(x)$ is constant or balanced later through interference.

- Reminder: The **phase** of a quantum state
     refers to whether an amplitude is positive or negative (or complex).<br>
    Two different quantum states  with the same probablisitic measrements but different phases might look like this:

    - $|\psi_1⟩ = \frac{1}{\sqrt{2}}(|0⟩ + |1⟩)$  
    - $|\psi_2⟩ = \frac{1}{\sqrt{2}}(|0⟩ - |1⟩)$



<br>



#### 3.4 Add the oracle (balanced for this example)

In quantum algorithms like Deutsch–Jozsa, an **oracle** is a special subcircuit that simulates the behavior of our function $f(x)$.

You can think of it as a **black box** that applies a transformation to the quantum state based on $f(x)$.  
It doesn't reveal the actual function outputs — instead, it uses quantum gates to **modify the state in a way that reflects whether $f(x)$ is 0 or 1**.

**In this example, we, the programmers, are the ones who define the oracle.**  
We choose a specific function $f(x)$ (in this case, a balanced one) and simulate its effect.
This lets us test how the Deutsch–Jozsa algorithm responds to different kinds of functions, ie, the oracle can be thought of as a stub in our algorithm.

One way to implement a balanced $f(x)$ is to apply the equivalent of an XOR to our $q_0$ and $q_1$, such that half our inputs will have an output of 1 and the other half 0.  The result of our output does not change out input qubits, and instead affects our ancillary $q_2$. The oracle acts on a combined state of the input |x⟩ and ancilla qubit |y⟩:  
$$
|x⟩|y⟩ \rightarrow |x⟩|y \oplus f(x)⟩
$$


In [ ]:
qc.cx(0, 2)  # Controlled-NOT: flips q2 if q0 is 1
qc.cx(1, 2)  # Controlled-NOT: flips q2 if q1 is 1

This means:
- If $f(x) = 0$, the oracle does nothing.
- If $f(x) = 1$, it attempt to bit flip the second qubit (`y`, our ancilla).

##### **Here's the key idea:** 
We have previously prepared our the ancilla qubit $q_2$ in the special quantum state  $|-\rangle$ : 
$$
\frac{|0⟩ - |1⟩}{\sqrt{2}} \quad 
$$

When the oracle evaluates $f(x) = 1$, it attempts to **bit flip** the ancilla using an X gate (the quantum NOT gate).  
Normally, X flips $|0⟩$ to $|1⟩$ and vice versa. But when applied to the $|-\rangle$ state, we get:

$$
X \left( \frac{|0⟩ - |1⟩}{\sqrt{2}} \right) = \frac{|1⟩ - |0⟩}{\sqrt{2}} = -\left( \frac{|0⟩ - |1⟩}{\sqrt{2}} \right) = -|-\rangle
$$

So the ancilla looks the same, but the entire system's state has gained a **global minus sign**.


Because the oracle applied that X gate conditionally based on the input $|x⟩$ , this global phase becomes **attached to the $|x⟩$ state** that caused it, ie the effect is passed **back** to the input.

This is what we call **phase kickback**.


Thus, even though the ancilla qubit itself doesn't visibly change or store $f(x)$, its interaction with the input qubits **leaves behind a subtle phase flip**  
in the **superposition of the input states $q_0$ and $q_1$** such that they become: 
      $$
      \frac{1}{2} (|00⟩ - |01⟩ - |10⟩ + |11⟩)  $$              


#### 3.5 Apply Hadamards to input qubits 

Now that the phase information from the oracle is encoded into the input qubits,  
we apply **Hadamard gates again** to $q_0$ and $q_1$.

The purpose of this second round of Hadamards is to **interfere** the amplitudes of the different basis states.

**Applying Hadamard gates**:

Each Hadamard acts like a "spreader", turning each qubit into a superposition: 
- $|0⟩$ becomes $\frac{1}{\sqrt{2}} (|0⟩ + |1⟩)$
- $|1⟩$ becomes $\frac{1}{\sqrt{2}} (|0⟩ - |1⟩)$


In [ ]:
qc.h([0,1])

Thus, applying $H \otimes H$ (tensor product) to each 2-qubit basis state:

- $|00⟩$ spreads into $(|00⟩ + |01⟩ + |10⟩ + |11⟩)$ with coeeficint of $\frac{1}{2}$
- $|01⟩$ spreads into $(|00⟩ - |01⟩ + |10⟩ - |11⟩)$ with coeeficint of $\frac{1}{2}$
- $|10⟩$ spreads into $(|00⟩ + |01⟩ - |10⟩ - |11⟩)$ with coeeficint of $\frac{1}{2}$
- $|11⟩$ spreads into $(|00⟩ - |01⟩ - |10⟩ + |11⟩)$ with coeeficint of $\frac{1}{2}$

* Tensor product is where we multiply every term in the first bracket by every term in the second bracket.
  
---

After applying $H \otimes H$, the full state becomes:

$$
\frac{1}{2} \left(
\frac{1}{2} (|00⟩ + |01⟩ + |10⟩ + |11⟩)
- \frac{1}{2} (|00⟩ - |01⟩ + |10⟩ - |11⟩)
- \frac{1}{2} (|00⟩ + |01⟩ - |10⟩ - |11⟩)
+ \frac{1}{2} (|00⟩ - |01⟩ - |10⟩ + |11⟩)
\right)
$$



Expanding inside the parentheses:

$$
= \frac{1}{2} \times \frac{1}{2} \left[
(|00⟩ + |01⟩ + |10⟩ + |11⟩)
- (|00⟩ - |01⟩ + |10⟩ - |11⟩)
- (|00⟩ + |01⟩ - |10⟩ - |11⟩)
+ (|00⟩ - |01⟩ - |10⟩ + |11⟩)
\right]
$$

Grouping like terms:

$$
= \frac{1}{4} \left[
(+1 -1 -1 +1)|00⟩ +
(+1 +1 -1 -1)|01⟩ +
(+1 -1 +1 -1)|10⟩ +
(+1 -1 -1 +1)|11⟩
\right]
$$

Simplifying each coefficient:

- For $|00⟩$: $(+1 -1 -1 +1) = 0$
- For $|01⟩$: $(+1 +1 -1 -1) = 0$
- For $|10⟩$: $(+1 -1 +1 -1) = 0$
- For $|11⟩$: $(+1 -1 -1 +1) = 2$

Thus, the final state after multiplying the global $\frac{1}{4}$:

$$
= \frac{1}{2} |11⟩
$$


#### 3.6 Measure input qubits 

- If the measured bits are **00** ➔ the function is **constant**.
- If the measured bits are **anything else** (01, 10, or 11) ➔ the function is **balanced**.


In [ ]:
qc.measure([0, 1], [0, 1])     #measure quantum bits 0 and 1 into classical register bits 0 and 1 


#####  **Why does $00$ mean the function is constant?**

- If $f(x)$ is **constant**, the quantum amplitudes **all add up** constructively at $|00⟩$ after the second Hadamards.
- This leads to a **100% chance** of measuring $|00⟩$.
- If $f(x)$ is **balanced**, the amplitudes **cancel out** at $|00⟩$ by destructive interference (as we've demonstarted in 3.5), and you will measure something else.


#### STEP 4: Execute on hardware 

Note that this may take a couple of minutes to run, depending on the queue.

In [ ]:
sampler = Sampler(mode=backend)
    # Create a Sampler primitive linked to the selected backend
    # This will send the circuit to real quantum hardware for execution

transpiled_circuit = transpile(qc, backend)
    # Transpile the circuit to match the backend's specific gate set and connectivity
    # This also optimizes the circuit so the real device can run it properly

job = sampler.run([transpiled_circuit], shots=1024)
    # Submit the circuit to the backend for execution
    # shots=1024 means we will run the circuit 1024 times to get good measurement statistics
    # Notice how we pass the sampler an array. It expects an array of circuit/

result = job.result()[0]
    # Get the result of the job
    # result[0] because Sampler returns a list of results (one per circuit)

counts = result.join_data().get_counts()
    # Merge the raw shot data across all executions (join_data),
    # then format it into a classical {bitstring: count} dictionary (get_counts)
    # so that we can easily plot and analyze the measurement results.

plot_histogram(counts, title="Measurement Results")
plt.show()
    #Plot and show histogram of results

qc.draw('mpl')
    # Visualize the quantum circuit

### **Congratulations!**

You have run a quantum circuit on actual quantum hardware! 	＼(＾O＾)／

---

#### **Note on results**

##### 1. Why are there runs with 00 as the result?

Real quantum hardware is **noisy**.  
There are small imperfections like **gate errors**, **decoherence**, and **readout errors** that can cause incorrect results.  
In an ideal, noiseless system, a **balanced function** should never produce `00`.  
However, on real devices, a small number of `00` measurements are expected due to these unavoidable noise effects.

If most of your measurements are `11` and only a few are `00`, your experiment still worked as intended!



##### 2. How is the result supposed to be 100% `11` when its amplitude was only $\frac{1}{2}$?

Remember from our introduction:  
**Amplitude and probability are not the same.**

- The amplitude of $|11⟩$ after the second Hadamard gates is $\frac{1}{2}$.
- To find the probability of measuring $|11⟩$, we **square the amplitude**:

$$
\left( \frac{1}{2} \right)^2 = \frac{1}{4}
$$

This would suggest a **25% chance** if other states also survived.

**However**, in the Deutsch–Jozsa algorithm, after the second Hadamards:
- **Destructive interference** cancels out the amplitudes of $|00⟩$, $|01⟩$, and $|10⟩$.
- Only $|11⟩$ remains with a nonzero amplitude.

Because the total probability must **still add up to 1** (normalization), the surviving $|11⟩$ state automatically **inherits all the probability mass**.

Thus, you get **near 100% probability** of measuring `11`!


#####  Extra Note on Normalization

In quantum mechanics:
- **Normalization** is not something we manually force — it is **preserved automatically** because all quantum operations (like Hadamard and CNOT) are **unitary**.
- When interference cancels some states, the probability is **redistributed** among the remaining states to keep the total probability = 1.

You don't have to normalize manually when running Qiskit circuits — it’s built into how quantum gates work!


